In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 15:06:51.753270


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

Parameters:
- \(C_{of}\): Cost per barrel of olefin feed
- \(C_{ir}\): Cost per barrel of isobutane recycle
- \(C_{aar}\): Cost per thousand pounds of acid addition rate
- \(C_{im}\): Cost per barrel of isobutane makeup
- \(V_{ao}\): Value per barrel of alkylate yield with motor octane number

Decision Variables:
- \(x_{of}\): Olefin feed (barrels per day)
- \(x_{ir}\): Isobutane recycle (barrels per day)
- \(x_{aar}\): Acid addition rate (thousands of pounds per day)
- \(y_{ay}\): Alkylate yield (barrels per day)
- \(y_{im}\): Isobutane makeup (barrels per day)
- \(y_{as}\): Acid strength (weight per cent)
- \(y_{mon}\): Motor octane number
- \(y_{eio}\): External isobutane-to-olefin ratio
- \(y_{adf}\): Acid dilution factor
- \(y_{fpn}\): F-4 performance number

Objective Function:
Maximize Profit \(P\), where:
\[ P = V_{ao} \cdot y_{ay} \cdot y_{mon} - C_{of} \cdot x_{of} - C_{ir} \cdot x_{ir} - C_{aar} \cdot x_{aar} - C_{im} \cdot y_{im} \]

Regression formulas:
- \(y_{ay} = x_{of} \cdot (1.12 + 0.13167 \cdot y_{eio} - 0.00667 \cdot y_{eio}^2)\)
- \(y_{mon} = 86.35 + 1.098 \cdot y_{eio} - 0.038 \cdot y_{eio}^2 + 0.325 \cdot (y_{as} - 89)\)
- \(y_{adf} = 35.82 - 0.222 \cdot y_{fpn}\)
- \(y_{fpn} = -133 + 3 \cdot y_{mon}\)

Constraints:
1. Bounds on decision variables:
\[0 \leq x_{of} \leq \text{Upper limit}\]
\[0 \leq x_{ir} \leq \text{Upper limit}\]
\[0 \leq x_{aar} \leq \text{Upper limit}\]
\[0 \leq y_{im} \leq \text{Upper limit}\]
\[0 \leq y_{as} \leq 100\]
\[80 \leq y_{mon} \leq 100\]
\[1 \leq y_{eio} \leq \text{Upper limit}\]
\[0 \leq y_{adf} \leq \text{Upper limit}\]
\[0 \leq y_{fpn} \leq \text{Upper limit}\]

2. External isobutane-to-olefin ratio:
\[y_{eio} = \frac{x_{ir} + y_{im}}{x_{of}}\]

3. Acid strength:
\[y_{as} = \frac{93000 \cdot x_{aar}}{x_{aar} \cdot y_{adf} + 1000 \cdot x_{aar}}\]

4. Alkylate yield:
\[1.22 \cdot y_{ay} = x_{of} + y_{im}\]

Note: The upper limits for the decision variables should be determined based on the specific constraints of the alkylation process.

In [7]:
print(response_1_text)

Parameters:
- \(C_{of}\): Cost per barrel of olefin feed
- \(C_{ir}\): Cost per barrel of isobutane recycle
- \(C_{aar}\): Cost per thousand pounds of acid addition rate
- \(C_{im}\): Cost per barrel of isobutane makeup
- \(V_{ao}\): Value per barrel of alkylate yield with motor octane number

Decision Variables:
- \(x_{of}\): Olefin feed (barrels per day)
- \(x_{ir}\): Isobutane recycle (barrels per day)
- \(x_{aar}\): Acid addition rate (thousands of pounds per day)
- \(y_{ay}\): Alkylate yield (barrels per day)
- \(y_{im}\): Isobutane makeup (barrels per day)
- \(y_{as}\): Acid strength (weight per cent)
- \(y_{mon}\): Motor octane number
- \(y_{eio}\): External isobutane-to-olefin ratio
- \(y_{adf}\): Acid dilution factor
- \(y_{fpn}\): F-4 performance number

Objective Function:
Maximize Profit \(P\), where:
\[ P = V_{ao} \cdot y_{ay} \cdot y_{mon} - C_{of} \cdot x_{of} - C_{ir} \cdot x_{ir} - C_{aar} \cdot x_{aar} - C_{im} \cdot y_{im} \]

Regression formulas:
- \(y_{ay} = x_{of}

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

To solve the optimization problem using Pyomo, we first need to import the necessary libraries and define the parameters and decision variables. For this example, I will use some sample data for the upper limits and cost parameters. Please replace them with the actual values according to your specific problem.

```python
import pyomo.environ as pyo
from pyomo.core import Var, Objective, Constraint

# Sample data for upper limits and cost parameters
UpperLimit_of = 1000
UpperLimit_ir = 2000
UpperLimit_aar = 100
UpperLimit_im = 500
UpperLimit_eio = 10
UpperLimit_adf = 20
UpperLimit_fpn = 50

C_of = 100  # Cost per barrel of olefin feed
C_ir = 50  # Cost per barrel of isobutane recycle
C_aar = 200  # Cost per thousand pounds of acid addition rate
C_im = 70  # Cost per barrel of isobutane makeup
V_ao = 150  # Value per barrel of alkylate yield with motor octane number

# Create a concrete Pyomo model
model = pyo.ConcreteModel()

# Decision Variables
model.x_of = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_of))
model.x_ir = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_ir))
model.x_aar = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_aar))
model.y_ay = Var(within=pyo.NonNegativeReals)
model.y_im = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_im))
model.y_as = Var(within=pyo.NonNegativeReals, bounds=(0, 100))
model.y_mon = Var(within=pyo.NonNegativeReals, bounds=(80, 100))
model.y_eio = Var(within=pyo.NonNegativeReals, bounds=(1, UpperLimit_eio))
model.y_adf = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_adf))
model.y_fpn = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_fpn))

# Objective Function
def obj_rule(model):
    return V_ao * model.y_ay * model.y_mon - C_of * model.x_of - C_ir * model.x_ir - C_aar * model.x_aar - C_im * model.y_im

model.objective = Objective(rule=obj_rule, sense=pyo.maximize)

# Regression formulas
model.regression_1 = Constraint(expr=model.y_ay == model.x_of * (1.12 + 0.13167 * model.y_eio - 0.00667 * model.y_eio ** 2))
model.regression_2 = Constraint(expr=model.y_mon == 86.35 + 1.098 * model.y_eio - 0.038 * model.y_eio ** 2 + 0.325 * (model.y_as - 89))
model.regression_3 = Constraint(expr=model.y_adf == 35.82 - 0.222 * model.y_fpn)
model.regression_4 = Constraint(expr=model.y_fpn == -133 + 3 * model.y_mon)

# Constraints
model.constraint_1 = Constraint(expr=model.y_eio == (model.x_ir + model.y_im) / model.x_of)
model.constraint_2 = Constraint(expr=model.y_as == 93000 * model.x_aar / (model.x_aar * model.y_adf + 1000 * model.x_aar))
model.constraint_3 = Constraint(expr=1.22 * model.y_ay == model.x_of + model.y_im)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print(f"Profit: {pyo.value(model.objective)}")
print(f"x_of: {pyo.value(model.x_of)}")
print(f"x_ir: {pyo.value(model.x_ir)}")
print(f"x_aar: {pyo.value(model.x_aar)}")
print(f"y_ay: {pyo.value(model.y_ay)}")
print(f"y_im: {pyo.value(model.y_im)}")
print(f"y_as: {pyo.value(model.y_as)}")
print(f"y_mon: {pyo.value(model.y_mon)}")
print(f"y_eio: {pyo.value(model.y_eio)}")
print(f"y_adf: {pyo.value(model.y_adf)}")
print(f"y_fpn: {pyo.value(model.y_fpn)}")
```

This code sets up the optimization problem and solves it using the IPOPT solver. The results are then printed. Replace the sample data with actual values according to your problem and update the solver if needed.

In [11]:
print(response_2_text)

To solve the optimization problem using Pyomo, we first need to import the necessary libraries and define the parameters and decision variables. For this example, I will use some sample data for the upper limits and cost parameters. Please replace them with the actual values according to your specific problem.

```python
import pyomo.environ as pyo
from pyomo.core import Var, Objective, Constraint

# Sample data for upper limits and cost parameters
UpperLimit_of = 1000
UpperLimit_ir = 2000
UpperLimit_aar = 100
UpperLimit_im = 500
UpperLimit_eio = 10
UpperLimit_adf = 20
UpperLimit_fpn = 50

C_of = 100  # Cost per barrel of olefin feed
C_ir = 50  # Cost per barrel of isobutane recycle
C_aar = 200  # Cost per thousand pounds of acid addition rate
C_im = 70  # Cost per barrel of isobutane makeup
V_ao = 150  # Value per barrel of alkylate yield with motor octane number

# Create a concrete Pyomo model
model = pyo.ConcreteModel()

# Decision Variables
model.x_of = Var(within=pyo.NonNegativeR

### Code Executability

In [12]:
import pyomo.environ as pyo
from pyomo.core import Var, Objective, Constraint

# Sample data for upper limits and cost parameters
UpperLimit_of = 1000
UpperLimit_ir = 2000
UpperLimit_aar = 100
UpperLimit_im = 500
UpperLimit_eio = 10
UpperLimit_adf = 20
UpperLimit_fpn = 50

C_of = 100  # Cost per barrel of olefin feed
C_ir = 50  # Cost per barrel of isobutane recycle
C_aar = 200  # Cost per thousand pounds of acid addition rate
C_im = 70  # Cost per barrel of isobutane makeup
V_ao = 150  # Value per barrel of alkylate yield with motor octane number

# Create a concrete Pyomo model
model = pyo.ConcreteModel()

# Decision Variables
model.x_of = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_of))
model.x_ir = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_ir))
model.x_aar = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_aar))
model.y_ay = Var(within=pyo.NonNegativeReals)
model.y_im = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_im))
model.y_as = Var(within=pyo.NonNegativeReals, bounds=(0, 100))
model.y_mon = Var(within=pyo.NonNegativeReals, bounds=(80, 100))
model.y_eio = Var(within=pyo.NonNegativeReals, bounds=(1, UpperLimit_eio))
model.y_adf = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_adf))
model.y_fpn = Var(within=pyo.NonNegativeReals, bounds=(0, UpperLimit_fpn))

# Objective Function
def obj_rule(model):
    return V_ao * model.y_ay * model.y_mon - C_of * model.x_of - C_ir * model.x_ir - C_aar * model.x_aar - C_im * model.y_im

model.objective = Objective(rule=obj_rule, sense=pyo.maximize)

# Regression formulas
model.regression_1 = Constraint(expr=model.y_ay == model.x_of * (1.12 + 0.13167 * model.y_eio - 0.00667 * model.y_eio ** 2))
model.regression_2 = Constraint(expr=model.y_mon == 86.35 + 1.098 * model.y_eio - 0.038 * model.y_eio ** 2 + 0.325 * (model.y_as - 89))
model.regression_3 = Constraint(expr=model.y_adf == 35.82 - 0.222 * model.y_fpn)
model.regression_4 = Constraint(expr=model.y_fpn == -133 + 3 * model.y_mon)

# Constraints
model.constraint_1 = Constraint(expr=model.y_eio == (model.x_ir + model.y_im) / model.x_of)
model.constraint_2 = Constraint(expr=model.y_as == 93000 * model.x_aar / (model.x_aar * model.y_adf + 1000 * model.x_aar))
model.constraint_3 = Constraint(expr=1.22 * model.y_ay == model.x_of + model.y_im)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print(f"Profit: {pyo.value(model.objective)}")
print(f"x_of: {pyo.value(model.x_of)}")
print(f"x_ir: {pyo.value(model.x_ir)}")
print(f"x_aar: {pyo.value(model.x_aar)}")
print(f"y_ay: {pyo.value(model.y_ay)}")
print(f"y_im: {pyo.value(model.y_im)}")
print(f"y_as: {pyo.value(model.y_as)}")
print(f"y_mon: {pyo.value(model.y_mon)}")
print(f"y_eio: {pyo.value(model.y_eio)}")
print(f"y_adf: {pyo.value(model.y_adf)}")
print(f"y_fpn: {pyo.value(model.y_fpn)}")

  - termination condition: infeasible
  - message from solver: Ipopt 3.12.13\x3a Converged to a locally infeasible point. Problem may be infeasible.


Profit: 186432.2629311357
x_of: 12.61519937762874
x_ir: 6.06917231265357
x_aar: 0.07807576210106873
y_ay: 15.705923210222535
y_im: 6.546026938842752
y_as: 91.17647057035654
y_mon: 80.0
y_eio: 1.0
y_adf: 20.0
y_fpn: 50.0


### Solution Correctness